<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto_Part 2</font></h1>

## Problem part 2:

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [2]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [3]:
soup = BeautifulSoup(source, 'xml')

In [4]:
table=soup.find('table')

In [66]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)
df

,Postalcode,Borough,Neighbourhood


In [67]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    #print(tr_cell)
    for td_cell in tr_cell.find_all('td'):
        td_data=[]
        td_data.append(td_cell.find('b').text.strip())
        td_data.append('Not assigned')
        td_data.append('Not assigned')
        td_data_idx = 1
        for td_ele in td_cell.find_all('a'):
            td_data[td_data_idx] = td_ele.text.strip()
            td_data_idx += 1
            if td_data_idx > 2:
                break
        #print(td_data)
        df.loc[len(df)] = td_data
        #df


In [68]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park


In [69]:
df=df[df['Borough']!='Not assigned']
df

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
...,...,...,...
160,M8X,Etobicoke,The Kingsway
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,Business reply mail,Not assigned
169,M8Y,Etobicoke,Old Mill


In [82]:
for index, row in df.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood'] = row['Borough']
    
#df[df[]=='Not assigned']=df['Borough']
df.head()


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


In [83]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [84]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [85]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [86]:
df_merge.drop_duplicates(inplace=True)

In [87]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)
df_merge.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Queen's Park,Queen's Park


In [88]:
df_merge.shape

(101, 3)

In [89]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [94]:
#geo_df = pd.read_csv('http://cocl.us/Geospatial_data') # 308 Permanent Redirect to -->
geo_df = pd.read_csv('https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv')


In [95]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [97]:
geo_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

In [98]:
geo_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
2,M1E,Scarborough,Guildwood,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
